# Бэггинг над решающими деревьями

Посмотрим, какие модели можно получить из деревьев с помощью их рандомизации

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('HR.csv')

target = 'left'
features = [c for c in data if c != target]
print(features)

X, y = data[features], data[target]

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


In [3]:
rnd_d3 = DecisionTreeClassifier(max_features=int(len(features) ** 0.5)) # Решающее дерево с рандомизацией в сплитах
d3 = DecisionTreeClassifier() # Обычное решающее дерево

Качество классификации решающим деревом с настройками по-умолчанию:

In [4]:
print("Decision tree:", cross_val_score(d3, X, y).mean())

('Decision tree:', 0.65217670200706801)


Бэггинг над решающими деревьями:

In [5]:
print("D3 bagging:", cross_val_score(BaggingClassifier(d3), X, y).mean())

('D3 bagging:', 0.71738247649529907)


Усредненная модель оказалась намного лучше. Оказывается, у решающих деревьев есть существенный недостаток - нестабильность получаемого дерева при небольших изменениях в выборке. Но бэггинг обращает этот недостаток в достоинство, ведь усредненная модель работает лучше, когда базовые модели слабо скоррелированы.

Изучив параметры DecisionTreeClassifier, можно найти хороший способ сделать деревья еще более различными - при построении каждого узла отбирать случайные max_features признаков и искать информативное разбиение только по одному из них.

In [6]:
print("Randomized D3 Bagging:", cross_val_score(BaggingClassifier(rnd_d3), X, y).mean())

('Randomized D3 Bagging:', 0.71711523638060948)


В среднем, качество получается еще лучше. Для выбора числа признаков использовалась часто применяемая на практике эвристика - брать корень из общего числа признаков. Если бы мы решали задачу регрессии - брали бы треть от общего числа.

In [7]:
print("Random Forest:", cross_val_score(RandomForestClassifier(), X, y).mean())

('Random Forest:', 0.72064875641795023)


In [8]:
print("Logistic Regression:", cross_val_score(LogisticRegression(), X, y).mean())

('Logistic Regression:', 0.62870531439621258)
